In [ ]:
import SimpleITK as sitk
import numpy as np
import os

In [ ]:
#help(sitk.WriteImage)

## Image Preprocessing:

In [ ]:
def image_preprocessing(image, ref_image):
    
    """
    Apply N4BiasFieldCorrection, Histogram Matching according to a reference image,
    Image Z-normalization (mu = 0 and variance = 1), and 8-bit rescaling (0,255) over
    the input MRI image.
    
    Argument:
    image -- SimpleItk supported MRI image (.nift, .nrrd, ...), with pixel type sitk.Float32, that
    will be corrected;
    ref-image -- SimpleItk supported MRI image (.nift, .nrrd, ...), with pixel type sitk.Float32, that
    was already N4BiasCorrected;
    
    Returns:
    prep_image -- the MRI image after this preprocessing pipeline;
    """
    print ("Running image_preprocessing() routine.")
    print (" ")
    
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    
    corrected_image = corrector.Execute(image)

    print ("N4BiasCorrection Done!")
    print (" ")
    
    hist_matcher = sitk.HistogramMatchingImageFilter()

    matched_image = hist_matcher.Execute(corrected_image, ref_image)
    
    print ("Histogram Matching Done!")
    print (" ")
    
    z_normalizer = sitk.NormalizeImageFilter()

    z_normalized_image = z_normalizer.Execute(matched_image)
    
    print ("Image Normalization Done!")
    print (" ")
    
    rescaler = sitk.RescaleIntensityImageFilter()
    rescaler.SetOutputMinimum(0)
    rescaler.SetOutputMaximum(255)    
    
    rescaled_image = rescaler.Execute(z_normalized_image) 
    
    print ("Image rescaling Done!")
    print (" ")
    
    prep_image = rescaled_image
    
    return prep_image    

In [ ]:
def make_images(group, image_type):
    
    """
    Read the images and labels files in folders and run feature extraction pipeline for a single patient
    lesion label. Since loading image, preprocessing, features extraction, features preprocessing,
    to final feature DataFrame for each greoup (MICCAI or WHM) and image_type (3DT1 or FLAIR).
    
    Argument:
    group -- WHM for Cerebrovascular patients, WHM Challenge patients, or MICCAI for Multiple Sclerosis, 
    MICCAI Challende Patients.
    
    Returns:
    final_dataframe - a DataFrame containing all extracted featuresfor both group accordint to image_type.
    There will be four csvs files at the end: WMH_3DT1, WMH_FLAIR, MICCAI_3DT1, MICCAI_FLAIR.
    """
    
    if group == 'MICCAI':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/MICCAI2016'
        image_3dt1 = '3DT1.nii.gz'
        image_flair = '3DFLAIR.nii.gz'
        prep_image_flair = 'prep_FLAIR.nrrd'
        prep_image_3dt1 = 'prep_3DT1.nrrd'

    if group == 'WMH':
        patient_dir = '/home/leonardo/Documents/Projeto-Bizu-MS/WHMChallenge'
        image_3dt1 = 'orig/3DT1.nii.gz'
        image_flair = 'orig/FLAIR.nii.gz'
        prep_image_flair = 'orig/prep_FLAIR.nrrd'
        prep_image_3dt1 = 'orig/prep_3DT1.nrrd'
     
    patient_count = 0

    patients = os.listdir(patient_dir)
    for patient in patients:
        
        print("Patient " + str(patient_count))
        print(' ')
        
        # Creating patient folder path
        patient_folder = os.path.join(patient_dir,patient)
        print(patient_folder)
        print(' ')

        if image_type == "FLAIR":
            image_file = os.path.join(patient_folder, image_flair)
            prep_image_name = os.path.join(patient_folder, prep_image_flair)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image-Preprocessing-and-Radiomic-Extraction/Histogram-Matching-Reference-Images/MICCAI-01016SACH-3DFLAIR.nrrd'
        
        elif image_type == "3DT1":
            image_file = os.path.join(patient_folder, image_3dt1)
            prep_image_name = os.path.join(patient_folder, prep_image_3dt1)
            ref_image_file = '/home/leonardo/Documents/Projeto-Bizu-MS/Classification-of-MRI-Hiperintense-Brain-Lesions/Image-Preprocessing-and-Radiomic-Extraction/Histogram-Matching-Reference-Images/MICCAI-01016SACH-3DT1.nrrd'
        
        if not os.path.isfile(image_file):
            print('OOOOOOOOOOOOOO - Images not found! - XXXXXXXXXXXX')
            print(' ')
            print(image_file)
            print(' ')
            
            continue
        
        print(image_file)
        print(' ')
        print(ref_image_file)
        print(' ')
        print(prep_image_name)
        print(' ')
        
        image = sitk.ReadImage(image_file, sitk.sitkFloat32)
        ref_image = sitk.ReadImage(ref_image_file, sitk.sitkFloat32)

        prep_image = image_preprocessing(image, ref_image)
        
        sitk.WriteImage(prep_image, prep_image_name)
        
        patient_count += 1
        
    return 

### Calling make images for both groups, both image types 

In [ ]:
make_images('WMH','3DT1')
make_images('WMH','FLAIR')

In [ ]:
make_images('MICCAI','3DT1')
make_images('MICCAI','FLAIR')